In [ ]:
!pip install kaggle==1.5.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73027 sha256=e3cb0fceb2b9e2fb585b272cfa14c1ba561b383f8720b7b764b42752b518d7ae
  Stored in directory: /root/.cache/pip/wheels/70/0c/e6/79103212a102e78b8453691b905f48000219574ba7137e7207
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.14
    Uninstalling kaggle-1.6.14:
      Successfully uninstalled kaggle-1.6.14


In [ ]:
pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ozzygomes","key":"8fe22947721bd006058b752c7b3d9955"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d augustop/portuguese-tweets-for-sentiment-analysis

 99% 223M/224M [00:03<00:00, 100MB/s] 
100% 224M/224M [00:03<00:00, 70.2MB/s]


In [ ]:
!unzip /content/portuguese-tweets-for-sentiment-analysis.zip


Archive:  /content/portuguese-tweets-for-sentiment-analysis.zip
  inflating: NoThemeTweets.csv       
  inflating: TestDatasets/Test.csv   
  inflating: TestDatasets/Test3classes.csv  
  inflating: TestDatasets/TestTema.csv  
  inflating: TrainingDatasets/Train100.csv  
  inflating: TrainingDatasets/Train200.csv  
  inflating: TrainingDatasets/Train300.csv  
  inflating: TrainingDatasets/Train3Classes.csv  
  inflating: TrainingDatasets/Train400.csv  
  inflating: TrainingDatasets/Train50.csv  
  inflating: TrainingDatasets/Train500.csv  
  inflating: TrainingDatasets/TrainTema.csv  
  inflating: TweetsNeutralHash.csv   
  inflating: TweetsNeutralNews.csv   
  inflating: TweetsWithTheme.csv     
  inflating: testdatasets/Test.csv   
  inflating: testdatasets/Test3classes.csv  
  inflating: testdatasets/TestTema.csv  
  inflating: trainingdatasets/Train100.csv  
  inflating: trainingdatasets/Train200.csv  
  inflating: trainingdatasets/Train300.csv  
  inflating: trainingdatasets/Train3

###Carregando os dados

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from torch.utils.data import DataLoader, Dataset
from sklearn import preprocessing
from tqdm import tqdm

In [ ]:
#flags de controle e hiperparametros
MAX_LENGTH = 512

TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1
BATCH_SIZE = 16

#CPU/GPU
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

In [ ]:
print("A unidade de processamento é: ", device)

A unidade de processamento é:  cuda:0


In [ ]:
df = pd.read_csv('/content/NoThemeTweets.csv')

In [ ]:
df.shape

(785814, 5)

In [ ]:
df['sentiment'].value_counts()

sentiment
Negativo    522707
Positivo    263107
Name: count, dtype: int64

como há muitos sentimentos negativos começarei com uma redução dos tweets negativos da base

In [ ]:
#contando o numero de sentimentos positivos e negativos
positive_count = df['sentiment'].value_counts()['Positivo']
negative_count = df['sentiment'].value_counts()['Negativo']

# calculando quantos negativos terie que remover
samples_to_remove = negative_count - positive_count

# randomicamente selecionando os negativos para serem removidos
negative_indices = df[df['sentiment'] == 'Negativo'].index
indices_to_remove = np.random.choice(negative_indices, samples_to_remove, replace=False)

# criando um novo dataaframe sem o excesso de amostras negativoas
df = df.drop(indices_to_remove)

In [ ]:
df['sentiment'].value_counts()

sentiment
Positivo    263107
Negativo    263107
Name: count, dtype: int64

In [ ]:
df.columns

Index(['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used'], dtype='object')

#BERT PORTUGUESE

###Tokenização dos dados

Nossas palavras precisam ser inseridas no modelo como números e, neste caso, usaremos o Tokenizer do próprio BERT, que utiliza uma técnica chamada Byte Pair Encoding por baixo dos panos.

In [ ]:

tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

  'df_tokenized' sera um dicionario com a keys ['input_ids', token_type_ids', 'attention_mask'] <p>
  'input_ids' -> instancias tokenizadas <p>
  'token_type_ids' ->  mascara usada em tarefas de classificação de frases em pares (sera dascartada nesta task) <p>
  'attention_maskk' -> mascara de atençao para o modelo os tokens de padding [PAD]

In [ ]:
df_tokenized = tokenizer.batch_encode_plus(df['tweet_text'], return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH)

Veriificando formato do dataset

In [ ]:
print(df_tokenized["input_ids"].shape, df_tokenized["attention_mask"].shape)

torch.Size([526214, 358]) torch.Size([526214, 358])


Definindo X e y <p>
[0, DATASET_LEN, MAX_LENGTH] = input_ids <p>
[1, DATASET_LEN, MAC_LENGTH] = attention_mask

Transformando o dftokenized em uma matrix com o trch stack

In [ ]:
X = torch.stack([df_tokenized["input_ids"], df_tokenized["attention_mask"]], dim=0)

df['sentiment'] = df['sentiment'].replace({'Negativo': 0, 'Positivo': 1})
y = torch.Tensor(df['sentiment'].to_numpy())

###Dataloader

Um Dataloader é o construtor de datasets do PyTorch. Vamos definir uma classe que será consumida pelo Dataloader quando alimentarmos o modelo com os dados durante o treino

In [ ]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.X = self.X.to(device)

        self.y = y
        self.y = self.y.to(device)

        self.len = len(y)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[:, idx], self.y[idx]

Instanciando os loaders de treino, validação e teste

In [ ]:
dataset = TextDataset(X, y)

#calculando o numero de instacias que devem ter em cada split
num_train_instances = int(np.round(dataset.len * TRAIN_RATIO))
num_val_instances = int(np.round(dataset.len * VAL_RATIO))
num_test_instances = int(np.round(dataset.len * TEST_RATIO))

print(f"Treino: {num_train_instances}, Validação: {num_val_instances}, Teste: {num_test_instances}")

train_split, val_split, test_split = torch.utils.data.random_split(dataset, [num_train_instances, num_val_instances, num_test_instances])

train_loader = torch.utils.data.DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_split, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_split, batch_size=BATCH_SIZE, shuffle=True)


Treino: 368350, Validação: 105243, Teste: 52621


###Setup do Treino

In [ ]:
epochs  = 40
steps_per_epoch = 200
epoch_validation_samples = 50

model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device)
for param in model.base_model.parameters():
    param.requires_grad = False

loss_func = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())

acc_calc = lambda output, labels: (labels == output.argmax(axis=1)).sum()

scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.9997)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###Treino

In [ ]:
apoch_metadata = []

for i in  range(epochs):
  num_train_examples = 0
  num_val_examples = 0

  train_hits = 0
  val_hits = 0

  train_bar = tqdm(total=steps_per_epoch, desc=f'Train', unit='steps', position=0, leave=True)
  val_bar = tqdm(total=epoch_validation_samples, desc=f'Val', unit='samples', position=0, leave=True)

  for batch_number, (features, labels) in enumerate(train_loader):

      train_running_loss = 0

      model.train()

      input_ids, input_masks = features[:, 0, :], features[:, 1, :]

      #loss, logits = model(input_ids, input_masks, labels=labels.long())
      var_temp = model(input_ids, input_masks, labels=labels.long())
      loss, logits = var_temp[0], var_temp[1]

      optim.zero_grad()
      loss.backward()
      optim.step()

      train_running_loss += loss.item()

      softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
      train_hits += acc_calc(softmax_predictions, labels)

      # Updating our display bar
      train_bar.update(1)

      num_train_examples += features.shape[0]

      scheduler.step()

      # Breaking after a certain amount of steps in the current epoch...
      if (batch_number + 1) % steps_per_epoch == 0:
          train_bar.close()
          break


  for batch_number, (features, labels) in enumerate(val_loader):
      with torch.no_grad():
          val_running_loss = 0

          model.eval()

          input_ids, input_masks = features[:, 0, :], features[:, 1, :]

          #loss, logits = model(input_ids, input_masks, labels=labels.long())
          var_temp = model(input_ids, input_masks, labels=labels.long())
          loss, logits = var_temp[0], var_temp[1]

          val_running_loss += loss.item()

          softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
          val_hits += acc_calc(softmax_predictions, labels)

          num_val_examples += features.shape[0]

          # Updating our display bar
          val_bar.update(1)

          # Breaking after a certain amount of steps in the current epoch....
          if (batch_number + 1) % epoch_validation_samples == 0:
              val_bar.close()
              break



  train_acc = torch.true_divide(train_hits, num_train_examples)
  val_acc = torch.true_divide(val_hits, num_val_examples)


  print(f"EPOCH SUMMARY -  {i + 1} \t Train loss: {train_running_loss} \t Train Acc: {train_acc} \t Val loss: {val_running_loss} \t Val Acc: {val_acc}")


Val: 100%|██████████| 50/50 [01:19<00:00,  1.59s/samples]


EPOCH SUMMARY -  1 	 Train loss: 0.5550014972686768 	 Train Acc: 0.6543749570846558 	 Val loss: 0.6535775661468506 	 Val Acc: 0.6662499904632568


Val: 100%|██████████| 50/50 [01:24<00:00,  1.70s/samples]


EPOCH SUMMARY -  2 	 Train loss: 0.5588162541389465 	 Train Acc: 0.7190625071525574 	 Val loss: 0.5427734851837158 	 Val Acc: 0.7862499952316284


Val: 100%|██████████| 50/50 [01:24<00:00,  1.70s/samples]


...

EPOCH SUMMARY -  38 	 Train loss: 0.5472568869590759 	 Train Acc: 0.8103125095367432 	 Val loss: 0.5497959852218628 	 Val Acc: 0.8337500095367432


Val: 100%|██████████| 50/50 [01:24<00:00,  1.69s/samples]


EPOCH SUMMARY -  39 	 Train loss: 0.4977725148200989 	 Train Acc: 0.8056249618530273 	 Val loss: 0.3999141454696655 	 Val Acc: 0.8700000047683716


Val: 100%|██████████| 50/50 [01:24<00:00,  1.69s/samples]

EPOCH SUMMARY -  40 	 Train loss: 0.290787935256958 	 Train Acc: 0.80531245470047 	 Val loss: 0.330471396446228 	 Val Acc: 0.8549999594688416


In [ ]:
model.save_pretrained(f'epoch_{i}')

###treino com dados reais

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=2
)
# Carregar pesos do modelo treinado (ajuste conforme o caminho do seu arquivo)
model.load_state_dict(torch.load("model_weights_02.pt", map_location=device))
model.to(device)
model.eval()  # Colocar o modelo em modo de avaliação


In [ ]:
# Ler o CSV com tweets para teste
new_df = pd.read_csv('BurgerKingBR (1).csv')

# Verificar as colunas disponíveis
print(new_df.columns)

# Se necessário, renomear a coluna do texto do tweet para 'tweet_text'
# Certifique-se de que o nome da coluna corresponde ao do seu CSV
if 'text' in new_df.columns:
    new_df.rename(columns={'text': 'tweet_text'}, inplace=True)
elif 'tweet' in new_df.columns:
    new_df.rename(columns={'tweet': 'tweet_text'}, inplace=True)
# Caso contrário, ajuste conforme necessário

# Função de pré-processamento adequada para BERT
def preprocess_tweet(tweet):
    # Remover URLs, mencões e hashtags
    tweet = re.sub(r'http\S+|@\w+|#\w+', '', tweet)
    # Remover espaços extras
    tweet = tweet.strip()
    return tweet

# Aplicar o pré-processamento
new_df['tweet_text'] = new_df['tweet_text'].apply(preprocess_tweet)

# Extrair os textos dos tweets
tweets = new_df['tweet_text'].tolist()



In [ ]:
# Tokenizar os tweets
encodings = tokenizer(
    tweets,
    truncation=True,
    padding=True,
    max_length=128,  # Ajuste conforme necessário
    return_tensors='pt'
)

# Criar o dataset personalizado
class NewTweetsDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings.input_ids)



In [ ]:
# Criar o dataset e o DataLoader
new_tweets_dataset = NewTweetsDataset(encodings)
new_tweets_loader = DataLoader(new_tweets_dataset, batch_size=32, shuffle=False)

# Fazer predições
all_predictions = []

with torch.no_grad():
    for batch in new_tweets_loader:
        # Mover batch para o dispositivo
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_predictions.extend(preds.cpu().numpy())



In [ ]:
# Mapear as previsões para labels
label_map = {0: 'Negativo', 1: 'Positivo'}
predicted_labels = [label_map[pred] for pred in all_predictions]

# Adicionar as previsões ao DataFrame
new_df['predicted_sentiment'] = predicted_labels

# Exibir as primeiras linhas com as previsões
print(new_df[['tweet_text', 'predicted_sentiment']].head())

# Contar o número de mensagens positivas e negativas
predictions_bradesco = new_df['predicted_sentiment'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
unique_predictions, counts = np.unique(predictions_bradesco, return_counts=True)

fig = px.bar(x=unique_predictions, y=counts,
             labels={'x':'Predictions', 'y':'Count'},
             title='Distribuição das Predições Bradesco')

fig.show()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
unique_predictions, counts = np.unique(predictions_petrobras, return_counts=True)

fig = px.bar(x=unique_predictions, y=counts,
             labels={'x':'Predictions', 'y':'Count'},
             title='Distribuição das Predições Petrobrás')

fig.show()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
unique_predictions, counts = np.unique(predictions_BurgerKingBR, return_counts=True)

fig = px.bar(x=unique_predictions, y=counts,
             labels={'x':'Predictions', 'y':'Count'},
             title='Distribuição das Predições BurgerKingBR')

fig.show()